In [1]:
from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, datasets
from PIL import Image
import os
%matplotlib inline

In [2]:
data_transforms = {
    'train': transforms.Compose([transforms.RandomResizedCrop(224),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor()]),
    'test': transforms.Compose([transforms.Resize(256),
                               transforms.CenterCrop(224),
                               transforms.ToTensor()])
}
data_dir = os.getcwd() + '/' + 'frd'
face_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'test']}
dataloaders = {x: DataLoader(face_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'test']}
dataset_sizes = {x: len(face_datasets[x]) for x in ['train', 'test']}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
import time
import copy

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=5):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch: {}/{}'.format(epoch, num_epochs-1))
        print('-'*10)
        
        for phase in ['train', 'test']:
            if phase == 'train':
                scheduler.step()
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            running_corrects = 0
            
            for faces, labels in dataloaders[phase]:
                faces = faces.to(device)
                labels = labels.to(device)
                #print(face.shape)
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase=='train'):
                    outputs = model(faces)

                    _, preds = torch.max(outputs, 1)
                    #return (outputs, labels)
                    loss = criterion(outputs, labels)
                    
                    if phase=='train':
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item() * faces.shape[0]
                running_corrects += torch.sum(preds == labels.data)
                
        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]
        
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
        time_elapsed = time.time() - since
        print('Time elapsed {:.0f}m {:.0f}s'.format(time_elapsed//60, time_elapsed%60))
        
        if phase=='test' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            
        print()
        
    time_elapsed = time.time() - since
    print("Training completed in {:.0f}m {:.0f}s".format(time_elapsed//60, time_elapsed%60))
    print("Best acc: {:.4f}".format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model
                    
                
                

In [5]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 394)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [6]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=5)

Epoch: 0/4
----------
test Loss: 2.5036 Acc: 0.4160
Time elapsed 1m 27s

Epoch: 1/4
----------
test Loss: 0.5744 Acc: 0.8747
Time elapsed 2m 52s

Epoch: 2/4
----------
test Loss: 0.1249 Acc: 0.9879
Time elapsed 4m 17s

Epoch: 3/4
----------
test Loss: 0.0565 Acc: 0.9949
Time elapsed 5m 42s

Epoch: 4/4
----------
test Loss: 0.0417 Acc: 0.9955
Time elapsed 7m 8s

Training completed in 7m 8s
Best acc: 0.9955


In [7]:
torch.save(model_ft, 'frd/best.model')